In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Persiapan Awal

Cek spek nvidia di google colab

In [ ]:
!nvidia-smi

Tue Dec 19 04:00:21 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

ambil data direktori HOME atau default dari google colab

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content


## Install yolo

install ultralytics dan import yolo

In [ ]:
# Pip install method (recommended)

!pip install ultralytics==8.0.20

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.2/78.2 GB disk)


In [ ]:
from ultralytics import YOLO

from IPython.display import display, Image

## Download dan load Yolo Large Version

download dan load model yolo8 large version dalam bentuk ekstensi .pt

In [ ]:
# model = YOLO(f'{HOME}/yolov8x.pt')

install modul roboflow untuk mendownload dataset yang sudah dilabeli

In [ ]:
!pip install roboflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.5/68.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.9 MB/s eta 0:00:00
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.1.1
    Uninstalling pyparsing-3.1.1:
      Successfully uninstalled pyparsing-3.1.1
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.8.1.78
    Uninstalling opencv-python-headless-4.8.1.78:
      Successfully uninstal

### Download dataset dan hasil labelling dari roboflow

download dataset yang sudah dilabeli

In [ ]:
!pip install roboflow

In [ ]:
import os
import pandas as pd

project_dir = '/content/drive/MyDrive/Esyarat/'
dataset_dir = project_dir+'dataset'
list_dataset = os.listdir(dataset_dir)
list_dataset = [dataset_dir+'/'+i for i in list_dataset]
list_dataframe = []
for df in list_dataset:
  df_te = pd.read_csv(df)
  df_te['dataset'] = [df_te['dataset'][i] if len(str(df_te['dataset'][i]).split("/")) > 2 else None for i in range(len(df_te))]
  df_te = df_te.dropna().reset_index(drop=True)
  df_te['workspace'] = [(str(df_te['dataset'][i]).split("/"))[1] for i in range(len(df_te))]
  df_te['project'] = [(str(df_te['dataset'][i]).split("/"))[2] for i in range(len(df_te))]
  df_te['status'] = ""
  list_dataframe.append(df_te)
  df_te.to_csv(df, index=False)

In [ ]:
import os
import shutil
from IPython import display
display.clear_output()
import ultralytics
ultralytics.checks()
from ultralytics import YOLO
from IPython.display import display, Image
from roboflow import Roboflow
import yaml
import subprocess

HOME = os.getcwd()

class LargeYoloModel:
    def __init__(self, nama_workspace, nama_projek, epoch):
        self.nama_workspace = nama_workspace
        self.nama_projek = nama_projek
        self.epoch = epoch

    def export_list_to_txt(nama_file, data_list):
      try:
        with open(nama_file, 'w') as file:
          for item in data_list:
            file.write(str(item) + '\n')
        print(f"List berhasil diekspor ke file {nama_file}.")
      except Exception as e:
        print(f"Error: {e}")

    def hapus_direktori(nama_direktori):
      try:
        shutil.rmtree(nama_direktori)
        print(f"Direktori {nama_direktori} berhasil dihapus.")
      except FileNotFoundError:
        print(f"Direktori {nama_direktori} tidak ditemukan.")
      except Exception as e:
        print(f"Error: {e}")

    def cari_file(direktori, nama_file):
      # Mencari file dengan prioritas temuan paling akhir
      temuan_terakhir = None
      for root, dirs, files in os.walk(direktori):
        if nama_file in files:
          temuan_terakhir = os.path.join(root, nama_file)

      return temuan_terakhir

    def pindahkan_file(sumber, tujuan, nama_baru=None):
      """
      Memindahkan file dari sumber ke tujuan dengan nama baru opsional.

      Parameters:
      - sumber (str): Path lengkap ke file yang akan dipindahkan.
      - tujuan (str): Direktori tujuan untuk memindahkan file.
      - nama_baru (str, optional): Nama baru untuk file setelah dipindahkan.

      Returns:
      - str: Path lengkap ke file yang sudah dipindahkan.
      """
      # Membuat direktori tujuan jika belum ada
      os.makedirs(tujuan, exist_ok=True)

      # Menentukan nama baru atau menggunakan nama asli
      nama_file = nama_baru if nama_baru else os.path.basename(sumber)

      # Menggabungkan path untuk tujuan pindah
      path_tujuan = os.path.join(tujuan, nama_file)

      # Memindahkan file
      shutil.move(sumber, path_tujuan)

      return path_tujuan

    def copy_file(dari, ke):
        try:
            shutil.copy(dari, ke)
            print(f"File berhasil disalin dari {dari} ke {ke}")
        except FileNotFoundError:
            print("File sumber tidak ditemukan.")
        except PermissionError:
            print("Tidak memiliki izin untuk menyalin file.")
        except Exception as e:
            print(f"Terjadi kesalahan: {e}")

    def run(self):
      nama_workspace = self.nama_workspace
      nama_projek = self.nama_projek
      LargeYoloModel.copy_file('/content/drive/MyDrive/Esyarat/yolov8x.pt', '/content/yolov8x.pt')
      model = YOLO(f'/content/yolov8x.pt')
      rf = Roboflow(api_key="jPzrt0coWYniVrkXMnTN") #API RoboFlow
      project = rf.workspace(nama_workspace).project(nama_projek)
      try:
        dataset = project.version(1).download("yolov8")
      except:
        print("gagal download")
        return "gagal download"

      dataYamlLocation = LargeYoloModel.cari_file(dataset.location, "data.yaml")
      with open(f"{dataYamlLocation}", "r") as stream:
        try:
          dict_yaml = yaml.safe_load(stream)
          # try:
          #   dict_yaml['names'] = [str(label)+f"_[{self.nama_workspace}/{nama_projek}]" for label in dict_yaml['names']]
          # except:
          #   pass
          dict_yaml['train'] = "../" + dict_yaml['train']
          dict_yaml['val'] = "../" + dict_yaml['val']
          LargeYoloModel.export_list_to_txt(f"/content/drive/MyDrive/Esyarat/label/{nama_workspace}_{nama_projek}_label.txt", dict_yaml['names'])

          # Menambahkan kode untuk menyimpan kembali ke file .yaml
          with open(f"{dataset.location}/modified_data.yaml", "w") as output_stream:
            yaml.dump(dict_yaml, output_stream, default_flow_style=False)
            print("Modified data saved to modified_data.yaml")
        except yaml.YAMLError as exc:
          print(exc)

      if dict_yaml['names'] != []:
        # Perintah YOLO yang ingin dieksekusi
        dataNewYamlLocation = LargeYoloModel.cari_file(dataset.location, "modified_data.yaml")
        perintah_yolo = f"yolo task=detect mode=train model={model} data={dataNewYamlLocation} epochs={self.epoch} imgsz=800 plots=True"
        print(perintah_yolo)

        # Menjalankan perintah shell dari dalam Python
        try:
          # subprocess.run(perintah_yolo, check=True, shell=True)
          !yolo task=detect mode=train model=/content/yolov8x.pt data={dataNewYamlLocation} epochs={self.epoch} imgsz=800 plots=True
        except subprocess.CalledProcessError as e:
          print(f"Error: {e}")

        # Contoh penggunaan
        direktori_cari = "runs"
        file_confusion_matrix = LargeYoloModel.cari_file(direktori_cari, "confusion_matrix.png")
        file_results = LargeYoloModel.cari_file(direktori_cari, "results.png")
        file_best_pt = LargeYoloModel.cari_file(direktori_cari, "best.pt")

        try:
          # Perintah YOLO yang ingin dieksekusi
          perintah_yolo = f"yolo task=detect mode=val model={file_best_pt} data={dataNewYamlLocation}"

          # Menjalankan perintah shell dari dalam Python
          try:
            !yolo task=detect mode=val model={file_best_pt} data={dataNewYamlLocation}
          except subprocess.CalledProcessError as e:
            print(f"Error: {e}")

          sumber_file = file_best_pt
          direktori_tujuan = "/content/drive/MyDrive/Esyarat"
          nama_file_baru = "yolov8x.pt"

          path_tujuan = LargeYoloModel.pindahkan_file(sumber_file, direktori_tujuan, nama_file_baru)
          print(f"File dipindahkan ke: {path_tujuan}")

          LargeYoloModel.hapus_direktori(dataset.location)
          LargeYoloModel.hapus_direktori("runs")

          print("berhasil")
          return "berhasil"
        except:
          print("tidak ada file best.pt")
        return "tidak ada file best.pt"
      else:
        print("label kosong")
        return "label kosong"



Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.5/78.2 GB disk)


In [ ]:
project_dir

'/content/drive/MyDrive/Esyarat/'

In [ ]:
list_dataset

['/content/drive/MyDrive/Esyarat/dataset/bisindo_roboflow_0_500.csv']

In [ ]:
dataset_list_with_status = {}
for dataset in list_dataset:
  dataset_list_with_status['dataset'] = dataset
  for i in range(len(dataset)):
    df = pd.read_csv(dataset)
    try:
      lym = LargeYoloModel(df['workspace'][i], df['project'][i], 10)
      res = lym.run()
      df['status'][i] = res
      df.to_csv(dataset)
      dataset_list_with_status['status'] = 'completed'
      continue
    except:
      dataset_list_with_status['status'] = 'empty'
      continue

File berhasil disalin dari /content/drive/MyDrive/Esyarat/yolov8x.pt ke /content/yolov8x.pt
loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.0.20, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to SIBI-1 in yolov8:: 100%|██████████| 302/302 [00:00<00:00, 4058.70it/s]


List berhasil diekspor ke file /content/drive/MyDrive/Esyarat/label/apollo-fikz-xpili_sibi-hh5if_label.txt.
Modified data saved to modified_data.yaml
yolo task=detect mode=train model=<ultralytics.yolo.engine.model.YOLO object at 0x795669a84610> data=/content/SIBI-1/modified_data.yaml epochs=10 imgsz=800 plots=True
Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/content/SIBI-1/modified_data.yaml, epochs=10, patience=50, batch=16, imgsz=800, save=True, cache=False, device=, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=False, val=True, save_json=False, save_hybrid=False, conf=0.001, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=ultralytics/assets


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.0.20, to fix: `pip install ultralytics==8.0.196`


...


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.0.20, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to bisindo-baru-1 in yolov8:: 100%|██████████| 4827/4827 [00:01<00:00, 4677.92it/s]


List berhasil diekspor ke file /content/drive/MyDrive/Esyarat/label/yoloooooo_bisindo-baru_label.txt.
Modified data saved to modified_data.yaml
yolo task=detect mode=train model=<ultralytics.yolo.engine.model.YOLO object at 0x795647ce3b20> data=/content/bisindo-baru-1/modified_data.yaml epochs=10 imgsz=800 plots=True
Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/content/bisindo-baru-1/modified_data.yaml, epochs=10, patience=50, batch=16, imgsz=800, save=True, cache=False, device=, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=False, val=True, save_json=False, save_hybrid=False, conf=0.001, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=ultralyt


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
# df

In [ ]:
# list_dataframe[0]

In [ ]:
# def run_train_large_yolo_model(df):
#   for i in range(len(df)):
#     lym = LargeYoloModel(df['workspace'][i], df['project'][i], 10)
#     lym.run()

In [ ]:
# lym = LargeYoloModel("krisna-wijaya-sxuzy", "kerusakan-jalan-yolov8", 10)
# lym.run()